In [1]:
import sys
print(sys.executable)
print(sys.path)

import numpy as np
import pandas as pd
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)

c:\Python312\python.exe
['c:\\Python312\\python312.zip', 'c:\\Python312\\DLLs', 'c:\\Python312\\Lib', 'c:\\Python312', '', 'C:\\Users\\Celeste\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Python312\\Lib\\site-packages']


ModuleNotFoundError: No module named 'numpy'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#피처 이름 파일 읽어오기
feature_name_df = pd.read_csv('../11장_data/UCI_HAR_Dataset/features.txt', sep = '\s+',header = None, names = ['index', 'feature_name'], engine = 'python')

In [ ]:
feature_name_df.head()

In [ ]:
feature_name_df.shape

In [ ]:
#index 제거하고, feature_name만 리스트로 저장
feature_name = feature_name_df.iloc[:, 1].values.tolist()

In [ ]:
feature_name[:5]

In [ ]:
X_train = pd.read_csv('../11장_data/UCI_HAR_Dataset/train/X_train.txt',delim_whitespace=True,header=None, encoding='latin-1')
X_train.columns = feature_name

X_test = pd.read_csv('../11장_data/UCI_HAR_Dataset/test/X_test.txt',delim_whitespace=True,header=None, encoding='latin-1')
X_test.columns = feature_name

Y_train = pd.read_csv('../11장_data/UCI_HAR_Dataset/train/y_train.txt', sep='\s+', header = None,names = ['action'], engine = 'python')
Y_test = pd.read_csv('../11장_data/UCI_HAR_Dataset/test/y_test.txt' , sep = '\s+', header = None,names = ['action'], engine = 'python')

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
X_train.head()

In [ ]:
print(Y_train['action'].value_counts())

In [ ]:
label_name_df = pd.read_csv('../11장_data/UCI_HAR_Dataset/activity_labels.txt', sep = '\s+', header = None, 
                            names= ['index', 'label'], engine = 'python')

In [ ]:
#index 제거하고, feature_name만 리스트로 저장
label_name = label_name_df.iloc[:, 1].values.tolist()

In [ ]:
label_name

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#결정 트리 분류 분석: 1) 모델 생성
dt_HAR = DecisionTreeClassifier(random_state=156)

In [ ]:
#결정 트리 분류 분석: 2) 모델 훈련
dt_HAR.fit(X_train, Y_train)

In [ ]:
#결정 트리 분류 분석: 3) 평가 데이터에 예측 수행 -> 예측 결과로 Y_predict 구하기
Y_predict = dt_HAR.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(Y_test, Y_predict)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

In [ ]:
print('결정 트리의 현재 하이퍼 매개변수: \n', dt_HAR.get_params())

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'max_depth' : [6, 8, 10, 12, 16, 20, 24]
}

grid_cv = GridSearchCV(dt_HAR, param_grid = params, scoring ='accuracy', cv = 5, return_train_score = True)
grid_cv.fit(X_train, Y_train)

In [ ]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df[['param_max_depth', 'mean_test_score', 'mean_train_score']]

In [ ]:
print('최고 평균 정확도: {0:.4f}, 최적 하이퍼 매개변수: {1}'.format(grid_cv.best_score_, grid_cv.best_params_))

In [ ]:
params = {
    'max_depth' : [8, 16, 20],
    'min_samples_split' : [8, 16, 24]
}

grid_cv = GridSearchCV(dt_HAR, param_grid = params, scoring ='accuracy', cv = 5, return_train_score = True)
grid_cv.fit(X_train, Y_train)

In [ ]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df[['param_max_depth', 'param_min_samples_split', 'mean_test_score', 'mean_train_score']]

In [ ]:
print('최고 평균 정확도: {0:.4f}, 최적 하이퍼 매개변수: {1}'.format(grid_cv.best_score_, grid_cv.best_params_))

In [ ]:
best_dt_HAR = grid_cv.best_estimator_
best_Y_predict = best_dt_HAR.predict(X_test)
best_accuracy = accuracy_score(Y_test, best_Y_predict)

print('best 결정 트리 예측 정확도: {0:.4f}'.format(best_accuracy))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
feature_importance_values = best_dt_HAR.feature_importances_
feature_importance_values_s = pd.Series(feature_importance_values, index = X_train.columns)

In [ ]:
feature_top10 = feature_importance_values_s.sort_values(ascending = False)[:10]

In [ ]:
plt.figure(figsize = (10, 5))
plt.title('Feature Top 10')
sns.barplot(x = feature_top10, y = feature_top10.index)
plt.show()

In [ ]:
from sklearn.tree import export_graphviz

#export_graphviz()의 호출 결과로 out_file로 지정된 tree.dot 파일 생성
export_graphviz(best_dt_HAR, out_file = "../11장_data/tree.dot", class_names = label_name, feature_names = feature_name, impurity = True, filled = True)

In [ ]:
import graphviz

#위에서 생성된 tree.dot 파일을 Graphviz가 읽어서 시각화
with open("../11장_data/tree.dot") as f:
    dot_graph = f.read()
    
graphviz.Source(dot_graph)